In [ ]:
import geopandas as gp
import pandas as pd
import searvey
import hvplot.pandas

In [ ]:
def get_meta() -> gp.GeoDataFrame:
    meta_web = searvey.get_ioc_stations().drop(columns=["lon", "lat"])
    meta_api = (
        pd.read_json(
            "http://www.ioc-sealevelmonitoring.org/service.php?query=stationlist&showall=all"
        )
        .drop_duplicates()
        .drop(columns=["lon", "lat"])
        .rename(columns={"Code": "ioc_code", "Lon": "lon", "Lat": "lat"})
    )
    merged = pd.merge(
        meta_web,
        meta_api[["ioc_code", "lon", "lat"]].drop_duplicates(),
        on=["ioc_code"],
    )
    return merged.drop(columns=["geometry"])

ioc_ = get_meta()
ioc_["is_ioc"] = True
ioc_['is_satellite'] = False
ioc_

In [ ]:
mycols = [str(i) for i in range(6)] # we expect 17 cols max in that file
stof2d = pd.read_csv(
    "https://polar.ncep.noaa.gov/stofs/data/stofs_2d_glo_elev_stat_v2_1_0",
    names=mycols, 
    sep="\t+|!", 
    header=None, 
    skiprows=1
)
stof2d['Info'] = stof2d.apply(lambda row: ' '.join(filter(None, row[2:])), axis=1)
stof2d['ID'] = stof2d['Info'].apply(lambda x: ' '.join(x.split()[:3]))
stof2d['Info'] = stof2d.apply(lambda row: row['Info'].replace(row['ID'], '').strip(), axis=1)
stof2d = stof2d.drop(columns=["2", "3", "4", "5"])
stof2d.rename(columns={"0": 'lon', "1": 'lat'}, inplace=True)
stof2d['is_satellite'] = stof2d.apply(lambda row: "SA" in row["ID"], axis=1)
stof2d['is_ioc'] = False
stof2d

In [ ]:
m = pd.merge(stof2d, ioc_,on=['lon', 'lat'], how='outer')
m

In [ ]:
m['is_ioc'] = m['is_ioc_x'].fillna(m['is_ioc_y'])
m['is_satellite'] = m['is_satellite_x'].fillna(m['is_satellite_y'])
m["id"] = m["ID"].fillna(m["ioc_code"])
m = m.drop(columns=['is_ioc_x', 'is_ioc_y', 'is_satellite_x', 'is_satellite_y'])
m

In [ ]:
m.hvplot.points(
    x='lon', y='lat', 
    c='is_ioc', 
    geo=True, 
    tiles = True, 
    cmap = "coolwarm", 
    hover_cols=["id"], 
).opts(
    width = 1800, 
    height=1000,
    title="False = STOFS list, True = IOC database", 
)

In [ ]:
m.hvplot.points(
    x='lon', y='lat', 
    c='is_satellite', 
    geo=True, 
    tiles = True, 
    cmap = "coolwarm", 
    hover_cols=["id"], 
).opts(
    width = 1800, 
    height=1000,
    title="Does the Point corresponds to setellite location? False / True", 
)